# Rascunho da implementação de detalhes do estabelecimento (REST)
*autor: André Costa (2019-03-09)* [https://www.linkedin.com/in/a-l-costa]


In [19]:
spark.sql('use geodata')

c_id = -103634676

concProfile = spark.sql('select a.id, a.codigo, a.nome, a.endereco, a.faixa_preco, \
    b.nome as bairro, b.populacao, b.dens_demo FROM concorrente as a INNER join bairro as b \
    ON a.id={} and a.bairro_id=b.codigo and isnull(b.end_date)'.format(c_id))
profile_data = concProfile.toPandas()
H1 = list(profile_data)
R1 = profile_data.values[0]
print(H1, '\n', R1)

['id', 'codigo', 'nome', 'endereco', 'faixa_preco', 'bairro', 'populacao', 'dens_demo'] 
 [-103634676 1710100459278383 'RT CHOPP & CIA'
 'Avenida Arymana, 954, 13056-440 Campinas' 1 'Ouro Verde' 141490
 4786.3740234375]


In [20]:
from pyspark.sql.types import *
from pyspark.sql.functions import UserDefinedFunction
import pandas as pd
import numpy as np

week_day = {0: 'dom', 1: 'seg', 2: 'ter', 3: 'qua', 4: 'qui', 5: 'sex', 6: 'sab'}

periodo = {0: 'manha', 1: 'tarde', 2: 'noite'}

def name_segments(day,p):
    return '_'.join([week_day[day], periodo[p]])

map_name_segments = UserDefinedFunction(name_segments, StringType())

fluxo = spark.sql('select AVG(a.ocorrencias) as fluxo_medio, b.dia_semana, b.periodo \
    FROM fluxo as a INNER join calendario as b \
    ON a.calendario_id=b.id WHERE concorrente_id={} group by b.dia_semana, b.periodo \
    order by b.dia_semana, b.periodo'.format(c_id))

fluxo = fluxo.withColumn('seg', map_name_segments(fluxo.dia_semana, fluxo.periodo)) \
    .drop('dia_semana', 'periodo')

fluxo_data = fluxo.toPandas().values.transpose()
H2 = fluxo_data[1].ravel()
R2 = fluxo_data[0].ravel()
print(H2, '\n', R2)

['dom_manha' 'dom_tarde' 'dom_noite' 'seg_manha' 'seg_tarde' 'seg_noite'
 'ter_manha' 'ter_tarde' 'ter_noite' 'qua_manha' 'qua_tarde' 'qui_manha'
 'qui_tarde' 'qui_noite' 'sex_manha' 'sex_tarde' 'sex_noite' 'sab_manha'
 'sab_tarde' 'sab_noite'] 
 [2.3333333333333335 5.333333333333333 5.0 5.0 1.5 1.0 1.0 1.5 1.0 1.0 6.0
 3.0 6.0 1.6666666666666667 1.0 1.75 1.0 3.3333333333333335 4.5
 1.6666666666666667]


In [28]:
H = ','.join(np.hstack((H1, H2)))
R = ','.join([str(x) for x in np.hstack((R1, R2))])
final_record = '\n'.join([H, R])
print(final_record)

id,codigo,nome,endereco,faixa_preco,bairro,populacao,dens_demo,dom_manha,dom_tarde,dom_noite,seg_manha,seg_tarde,seg_noite,ter_manha,ter_tarde,ter_noite,qua_manha,qua_tarde,qui_manha,qui_tarde,qui_noite,sex_manha,sex_tarde,sex_noite,sab_manha,sab_tarde,sab_noite
-103634676,1710100459278383,RT CHOPP & CIA,Avenida Arymana, 954, 13056-440 Campinas,1,Ouro Verde,141490,4786.3740234375,2.3333333333333335,5.333333333333333,5.0,5.0,1.5,1.0,1.0,1.5,1.0,1.0,6.0,3.0,6.0,1.6666666666666667,1.0,1.75,1.0,3.3333333333333335,4.5,1.6666666666666667
